## 1.1 IQR和3σ离群值检测

### 1.1.1 方法简介

**仅限于一维**数据$X=\{x_1, x_2, \cdots, x_n\}$的离群值检测。离群值检测是指识别数据集中的异常值，即与其他数据点显著不同的数据点。离群值检测的目标是识别这些异常值，以便在进一步分析中将其排除或进行其他处理。对于一维数据来说，IQR和3σ正态分布离群值检测是两种常用的方法，其中**IQR无需假设数据分布**，而**3σ则需要假设数据为正态分布**。

### 1.1.2 算法实现

#### (1) IQR异常值检测:

算法流程如下：
- 计算出IQR并且计算出上下界（以1.5倍IQR为标准）
- 判断数据是否在上下界之间，若不在则为异常值

其中相关公式如下：

- 计算四分位数：确定数据集的**第一四分位数**（Q1）和**第三四分位数**（Q3）。第一四分位数（Q1）是将数列升序排列后位于25%位置的数值，而第三四分位数（Q3）是位于75%位置的数值。
- 计算IQR（四分位距）：计算四分位距（IQR），它是Q3和Q1的差值，用于衡量数据的离散程度和异常值的界定基准，即：  
  $$IQR = Q3 - Q1$$


  
  
  
- 定义异常值的界限：根据IQR计算出数据的异常值界限。这通常涉及到确定“内部”范围，以辨别哪些数据点可以被认为是异常值。计算上界（upper bound）和下界（lower bound）的标准方法是：
  - **上界** = $Q3 + 1.5 * IQR$
  - **下界** = $Q1 - 1.5 * IQR$
这里的1.5是一个常用的乘数，但可以根据具体应用调整以识别更极端的异常值（例如，使用3倍IQR而不是1.5倍来定义更严格的界限）。

#### (2) 正态分布离群值检测：

算法流程如下：
- 计算出数据的均值$\mu$和标准差$\sigma$，从而得出上下界（以$3\sigma$原则为例）
- 判断数据是否在上下界之间，若不在则为异常值

其中相关公式如下：
- 计算均值$\mu$：计算数据集的均值$\mu$，即所有数据的平均值：
  $$\mu = \frac{1}{n}\sum_{i=1}^{n}x_i$$
- 计算标准差$\sigma$：计算数据集的标准差$\sigma$，即数据集中所有数据与均值的差值的平方和的平均值的平方根：
  $$\sigma = \sqrt{\frac{1}{n}\sum_{i=1}^{n}(x_i - \mu)^2}$$
- 用$3\sigma$来定义异常值的界限：计算上界（upper bound）和下界（lower bound）的标准方法是：
  - **上界** = $\mu + 3\sigma$
  - **下界** = $\mu - 3\sigma$

## 1.2 局部离群因子 (LOF)

### 1.2.1 方法简介

LOF算法是一种**基于密度**的离群点检测算法，它的基本思想是通过计算每个样本点的局部密度与其邻域样本点的局部密度之比来判断样本点是否为离群点。**适用于高维数据集**，对数据分布没有假设，对邻近点数量$k$**参数敏感**。

### 1.2.2 算法实现

LOF算法的实现步骤如下：

- (i) 计算每个样本点的k-距离（k-distance）；
- (ii) 计算每个样本点的局部可达密度（local reachability density）；
- (iii) 计算每个样本点的局部离群因子（local outlier factor）。
- (iv) 根据局部离群因子的数值，判断样本点是否为离群点。

相关公式如下：

- (i) k-距离：$k\_ distance(p) = d(p, o_k)$，其中，$o_k$是样本点p的第k个最近邻的样本点，$d(p, o_k)$是样本点p到样本点$o_k$的距离，取欧式距离，即：

  $$d(p, o_k) = \sqrt{\sum_{i=1}^{n}(p_i - o_{ki})^2}$$

  其中，n是样本点的特征维度，$p_i$是样本点p的第i个特征，$o_{ki}$是样本点$o_k$的第i个特征。

- (ii) 局部可达密度：
            
  $$lrd(p) = 1 / (\frac{\sum_{o \in N_k(p)} reach\_dist_k(p, o)}{|N_k(p)|})$$

  其中，
  - $lrd(p)$是样本点p的局部可达密度，值越大则密度越大。
  - $N_k(p)$是样本点p的k-邻域，取绝对值则为样本点p的k-邻域的样本点个数。
  - $reach\_ dist_k(p, o) = \max\{k\_ distance(o), d(p, o)\}$，是样本点$p$到样本点$o$的可达距离（这是由于**考虑了密度**，即若$p$到$o$的距离小于$o$的$k-$距离，**则认为这两点的密度相近**，因此取$o$的$k-$距离作为$p$到$o$的可达距离）
  - 注意$p$到$o$的可达距离一般不等于$o$到$p$的可达距离。
  
- (iii) 局部离群因子：$$LOF(p) = \frac{1}{|N_k(p)|}\sum_{o \in N_k(p)} \frac{lrd(o)}{lrd(p)}$$
  其中，
  - $LOF(p)$是样本点p的局部离群因子。
  - $\frac{lrd(o)}{lrd(p)}$是样本点$o$的局部可达密度与样本点$p$的**局部可达密度之比**，这个**比值越大**，说明样本$p$的密度相对于样本$o$的密度越小，样本$p$则**越可能是离群点**。

- (iv) 根据局部离群因子的大小，判断样本点是否为离群点，因此有： 
  - $LOF(p)$越大，样本点p越可能是离群点。
  - $LOF(p)$越小，样本点p越可能是正常点。

## 1.3 孤立森林 (IForest)

### 1.3.1 方法简介

孤立森林（Isolation Forest, 简称iForest）是一种**基于树的异常点检测算法**，它的核心思想是随机选择一个特征并随机选择一个切分值来递归地切分数据，直到数据点被孤立。由于异常点的数值属性通常与大多数数据点不同，因此它们更容易被孤立出来，也就是说，它们在树结构中的路径较短。**适用于连续性和离散性数据**，对大规模数据有很好的适应性，计算复杂度低。孤立森林算法特别**适合处理大规模数据集**，因为它的时间复杂度和空间复杂度相对较低，而且**不需要假设数据分布**，具有很好的通用性和适应性。

### 1.3.2 算法实现

孤立森林算法的实现步骤如下：

- (i) 从数据集中随机选择一部分样本构建孤立森林；
- (ii) 随机选择一个特征和该特征的最大值与最小值之间的一个切分值；
- (iii) 根据这个切分值切分数据集，生成左右子树；
- (iv) 重复(ii)和(iii)步骤，直到每个数据点都被孤立，或者达到树的最大高度；
- (v) 对每一个数据点，计算它在树中的路径长度；
- (vi) 计算每个数据点的平均路径长度，作为其异常分数；
- (vii) 根据异常分数判断数据点是否为异常点。

相关概念和公式如下：

- (i) 异常分数：给定一个样本点，其异常分数是基于该点被孤立的难易程度。如果一个样本在较少的切分下被孤立，那么它的异常分数较高，反之则较低。
  
  $$s(x, n) = 2^{-\frac{E(h(x))}{c(n)}}$$
  
  其中，
  - $s(x, n)$是样本点$x$的异常分数；
  - $E(h(x))$是样本点$x$在森林中所有树上路径长度的平均值；
  - $c(n)$是不包含点$x$的数据集的平均路径长度的归一化因子，用于比较不同大小数据集的异常分数；
  - $n$是样本总数。

- (ii) 路径长度$h(x)$：从根节点到达节点$x$所经过的边数。

- (iii) 归一化因子$c(n)$：对于含有$n$个数据点的数据集，其归一化因子可以通过以下公式计算：
  
  $$c(n) = 2H(n-1) - \frac{2(n-1)}{n}$$
  
  其中，$H(i)$是$i$的调和数，可以近似表示为$\ln(i) + 0.5772156649$（欧拉常数）。


## 1.4 密度聚类算法 (DBSCAN)

### 1.4.1 方法简介

DBSCAN算法是一种**基于密度**的聚类算法，通过连接高密度的样本点来形成聚类。它能够在带有噪声的数据集中发现任意形状的聚类，并且不需要预先指定聚类的数量。DBSCAN对于异常点具有良好的鲁棒性，**适用于发现具有复杂形状和大小的聚类**。算法的主要特点是依赖两个参数：邻域半径（$\epsilon$）和最小点数（MinPts），这两个参数直接影响聚类的结果，即**参数敏感**。DBSCAN能够有效地识别出数据集中的聚类结构，即使聚类的形状非常复杂或者数据中含有噪声。

### 1.4.2 算法实现

DBSCAN算法的实现步骤如下：

- (i) 为每个点定义$\epsilon$-邻域：点$p$的$\epsilon$-邻域包括点$p$以及与$p$距离不超过$\epsilon$的所有点。
- (ii) 标记核心点、边界点和噪声点：
  - 核心点：在点$p$的$\epsilon$-邻域内至少有MinPts个点（包括点$p$本身），则点$p$是一个核心点。
  - 边界点：在点$p$的$\epsilon$-邻域内少于MinPts个点，但是点$p$至少在一个核心点的$\epsilon$-邻域内。
  - 噪声点：既不是核心点也不是边界点的点。
- (iii) 形成聚类：
  - 从一个随机的核心点开始，通过递归地添加所有直接密度可达的核心点及其$\epsilon$-邻域内的所有点来形成一个聚类。
  - 一个边界点可以被多个聚类共享。
- (iv) 完成聚类：重复步骤(iii)直到所有的核心点都被访问，从而形成不同的聚类。

相关概念如下：

- **直接密度可达**：如果点$p$在点$q$的$\epsilon$-邻域内，并且点$q$是一个核心点，则称点$p$从点$q$直接密度可达。
- **密度可达**：如果存在一个点的序列$p_1, ..., p_n$，其中$p_1=q$且$p_n=p$，并且对于任意的$i$（$1 < i \leq n$），$p_i$从$p_{i-1}$直接密度可达，则称点$p$从点$q$密度可达。
- **密度相连**：如果存在一个点$o$，使得点$p$和点$q$都从点$o$密度可达，则称点$p$和点$q$密度相连。

## 1.5 自编码器 (AE)

### 1.5.1 方法简介

自编码器（Autoencoder, AE）是一种**无监督的学习算法**，主要用于数据的降维或特征学习。自编码器通过训练数据来学习一个表示（编码）用于数据压缩，同时尽可能减少与原始数据之间的差异。自编码器的结构通常包括两部分：编码器（Encoder）和解码器（Decoder）。编码器的作用是将输入数据转换成一个更低维度的编码，解码器的作用是从这个编码重新构建数据。理想情况下，解码后的输出与原始输入非常接近。自编码器在降维、特征提取、数据去噪等方面有着广泛的应用。

### 1.5.2 算法实现

自编码器的实现步骤通常如下：

- (i) **设计自编码器架构**：确定编码器和解码器的层次结构、激活函数、损失函数等。
- (ii) **训练自编码器**：使用训练数据集训练自编码器，通过优化算法（如梯度下降）最小化重建误差。
- (iii) **数据编码与重建**：使用训练好的自编码器对数据进行编码和重建，评估模型的性能。

相关概念：

- **编码器（Encoder）**：将输入数据映射到一个隐藏层，这个隐藏层可以被视为数据的一个压缩表示。
  
- **解码器（Decoder）**：将隐藏层的压缩表示映射回原始数据空间，尽可能恢复原始数据。
  
- **损失函数（Loss function）**：衡量重建数据与原始数据之间的差异，常见的损失函数有均方误差（MSE）等。
  
- **激活函数（Activation function）**：用于引入非线性，使得自编码器能够学习到复杂的数据表示。

自编码器的关键在于学习到的编码能够捕捉到数据中最重要的特征，而忽略掉噪声和不重要的细节。因此，自编码器特别适合于那些需要数据降维或特征提取的场景。此外，自编码器也可以通过一些变种（如稀疏自编码器、去噪自编码器等）来实现更特定的功能，例如数据去噪、异常检测等。